In [ ]:
import libtorrent as lt
import time

# List of high-traffic trackers to "poke" the internet
trackers = [
    "udp://tracker.opentrackr.org:1337/announce",
    "udp://9.rarbg.com:2810/announce",
    "udp://tracker.openbittorrent.com:6969/announce",
    "http://tracker.openbittorrent.com:80/announce",
    "udp://exodus.desync.com:6969",
    "udp://open.stealth.si:80/announce",
    "udp://tracker.torrent.eu.org:451/announce"
]

def download_torrent_boosted(magnet_hash):
    ses = lt.session()
    ses.listen_on(6881, 6891)
    
    # Append trackers to your hash to make it a "Loud" magnet link
    tracker_str = "".join([f"&tr={t}" for t in trackers])
    full_magnet = f"magnet:?xt=urn:btih:{magnet_hash}{tracker_str}"
    
    params = {'save_path': '/content/raw_data'}
    handle = lt.add_magnet_uri(ses, full_magnet, params)
    
    print(f"🧲 Attempting to find peers for {magnet_hash}...")
    
    start_time = time.time()
    while not handle.has_metadata():
        s = handle.status()
        # This will tell you if you are even seeing other computers
        print(f"Searching... | Peers: {s.num_peers} | Time: {int(time.time() - start_time)}s", end='\r')
        time.sleep(2)
        if time.time() - start_time > 300: # 5 minute limit for metadata
            print("\n❌ No peers found after 5 mins. The university server might be offline.")
            return None

    print("\n✅ Metadata Found! Starting Download...")
    # ... rest of the download logic ...
    return handle

# Try January again with the boost
download_torrent_boosted("4fd14d4c3d792e0b1c5cf6b1d9516c48ba6c4a24")